In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import SimpleITK as sitk
from tqdm import tqdm
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

from pathlib import Path
import pyrootutils

import torch.nn as nn
import hydra

import numpy as np
import torch
from omegaconf import  OmegaConf
from pytorch_lightning import LightningModule

from tqdm import tqdm
import pandas as pd
import src.utils.default as utils
import SimpleITK as sitk
sitk.ProcessObject_SetGlobalWarningDisplay(False)
from src.data.bvisa_dm import CS_Dataset

import SimpleITK as sitk

from src.data.bvisa_dm import CS_Dataset

import matplotlib.pyplot as plt

from monai.metrics import compute_dice, compute_hausdorff_distance, compute_iou

# Load segmentation model

In [2]:
CHKP = Path('/mrhome/vladyslavz/git/central-sulcus-analysis/logs_finetuning/CS2x_noSST_monaiUnet-fullFinetune/runs/2023-04-24_13-28-18/checkpoints/epoch-220-Esubj-0.4944.ckpt')

out_path = Path('/mrhome/vladyslavz/git/central-sulcus-analysis/data/via11/nobackup/segm_results/skull_stripped_images')

exp_name = CHKP.parent.parent.parent.parent.name
out_path = out_path / exp_name
out_path.mkdir(exist_ok=True, parents=True)
cgf_path = CHKP.parent.parent / '.hydra' / 'config.yaml'
finetune_cfg = OmegaConf.load(cgf_path)

segm_model: LightningModule = hydra.utils.instantiate(finetune_cfg.model)
# sst_ds = hydra.utils.instantiate(finetune_cfg.data)
# print(finetune_cfg.data)
segm_model = segm_model.load_from_checkpoint(CHKP).to('cuda')
segm_model = segm_model.eval()


Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
Attribute 'loss_function' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_function'])`.


# Load via validation images

In [3]:
via11DS = CS_Dataset('via11', 'mp2rage_skull_stripped',
                    'bvisa_CS', dataset_path='',
                     split='only_good',

                     preload=False,
                     resample=finetune_cfg.data.dataset_cfg.resample,
                     padd2same_size=finetune_cfg.data.dataset_cfg.padd2same_size)

In [4]:
experiment_results = []
for idx in tqdm(range(len(via11DS))):
    sample = via11DS[idx]
    target_1hot = torch.nn.functional.one_hot(sample['target'].unsqueeze(0), num_classes=2).permute(0, 4, 1, 2, 3)

    with torch.no_grad():
        out = segm_model.forward(sample['image'].unsqueeze(0).to('cuda'))
    out_bin = torch.argmax(torch.softmax(out, dim=1), dim=1).cpu()
    out_1hot = torch.nn.functional.one_hot(out_bin, num_classes=2).permute(0, 4, 1, 2, 3)

    dice = compute_dice(out_1hot, target_1hot, include_background=False)
    iou = compute_iou(out_1hot, target_1hot, include_background=False)
    hausdorff_distance = compute_hausdorff_distance(out_1hot, target_1hot, include_background=False)

    res = {'caseid': via11DS.caseids[idx],
           'dice': dice.item(),
           'iou': iou.item(),
           'hausdorff_distance': hausdorff_distance.item()
           }
    experiment_results.append(res)
experiment_results = pd.DataFrame(experiment_results)
experiment_results.to_csv(f'{out_path}/via11_metrics.csv')


100%|██████████| 101/101 [00:43<00:00,  2.30it/s]


In [5]:
# plt.imshow(out_hot[0, 1, 120, :, :], cmap='gray', alpha=0.5)
# plt.imshow(target_1hot[0, 1, 120, :, :], cmap='gray', alpha=0.5)
# plt.show()



# fig, axs = plt.subplots(1, 1, figsize=(5, 5))
# slice = 120
# axs.imshow(sample['image'][0, slice, :, :], cmap='gray', alpha=0.5)
# axs.imshow(sample['target'][slice, :, :], cmap='gray', alpha=0.5)
# plt.show()